In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np

In [5]:
# Вариант 1: Создание модели через список слоев (ИСПРАВЛЕНО)
model = keras.Sequential([
    Dense(128, activation='relu', name='hidden_1'),
    Dense(10, activation='softmax', name='output')
])

# Вариант 2: Постепенное добавление слоев (ИСПРАВЛЕНО)
model = keras.Sequential()
model.add(Dense(128, activation='relu', input_shape=(784,), name='hidden_1'))  # Запятая вместо точки
model.add(Dense(10, activation='softmax', name='output'))

print("Слои модели:")
print(model.layers)

Слои модели:
[<Dense name=hidden_1, built=True>, <Dense name=output, built=True>]


In [6]:
# Инициализация весов путем пропуска данных через модель
x = tf.random.uniform((1, 784), 0, 1)
y = model(x)

print("\nВеса модели после инициализации:")
for i, layer in enumerate(model.layers):
    print(f"Слой {i}: {len(layer.weights)} наборов весов")

# Вывод структуры модели
print("\nСтруктура модели:")
model.summary()


Веса модели после инициализации:
Слой 0: 2 наборов весов
Слой 1: 2 наборов весов

Структура модели:


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ hidden_1 (Dense)                     │ (None, 128)                 │         100,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Загрузка данных
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Нормализация данных
x_train = x_train / 255.0
x_test = x_test / 255.0

# Изменение формы данных
x_train = tf.reshape(tf.cast(x_train, tf.float32), [-1, 28*28])
x_test = tf.reshape(tf.cast(x_test, tf.float32), [-1, 28*28])

# Преобразование меток в one-hot encoding
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

print(f"\nФорма обучающих данных: {x_train.shape}")
print(f"Форма тестовых данных: {x_test.shape}")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step 

Форма обучающих данных: (60000, 784)
Форма тестовых данных: (10000, 784)


In [8]:
# Компиляция модели
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучение модели
print("\nНачало обучения...")
history = model.fit(x_train, y_train_cat, 
                   batch_size=32, 
                   epochs=5,
                   validation_data=(x_test, y_test_cat))

# Оценка модели
test_loss, test_acc = model.evaluate(x_test, y_test_cat)
print(f"\nТочность на тестовых данных: {test_acc:.4f}")


Начало обучения...
Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9271 - loss: 0.2574 - val_accuracy: 0.9574 - val_loss: 0.1439
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9663 - loss: 0.1123 - val_accuracy: 0.9716 - val_loss: 0.0898
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9766 - loss: 0.0771 - val_accuracy: 0.9698 - val_loss: 0.0911
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9819 - loss: 0.0578 - val_accuracy: 0.9708 - val_loss: 0.0969
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9865 - loss: 0.0442 - val_accuracy: 0.9772 - val_loss: 0.0755
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 726us/step - accuracy: 0.9772 - loss: 0.0755

Точность на тестовых данных: 0.9772


In [9]:
# Создание модели с несколькими выходами
model_multi = keras.Model(inputs=model.inputs,
                         outputs=[layer.output for layer in model.layers])

# Тестирование многовыходной модели
x_sample = tf.expand_dims(x_test[0], axis=0)
outputs = model_multi(x_sample)
original_output = model(x_sample)

print("\nСравнение выходов:")
print("Многовыходная модель:")
for i, output in enumerate(outputs):
    print(f"Слой {i}: форма {output.shape}")
    
print("\nОригинальная модель:")
print(f"Выход: форма {original_output.shape}")

# Проверка идентичности выходов
print(f"\nВыходы идентичны: {np.allclose(outputs[-1].numpy(), original_output.numpy())}")


Сравнение выходов:
Многовыходная модель:
Слой 0: форма (1, 128)
Слой 1: форма (1, 10)

Оригинальная модель:
Выход: форма (1, 10)

Выходы идентичны: True


In [10]:
# Модель только с первым скрытым слоем
model_truncated = keras.Model(inputs=model.inputs,
                             outputs=model.layers[0].output)

print("\nУрезанная модель (только первый слой):")
model_truncated.summary()

# Тестирование урезанной модели
truncated_output = model_truncated(x_sample)
print(f"\nВыход урезанной модели: форма {truncated_output.shape}")


Урезанная модель (только первый слой):


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ hidden_1 (Dense)                     │ (None, 128)                 │         100,480 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 100,480 (392.50 KB)

 Trainable params: 100,480 (392.50 KB)

 Non-trainable params: 0 (0.00 B)


Выход урезанной модели: форма (1, 128)


In [11]:
# Создание расширенной модели
model_extended = keras.Sequential([
    model,
    Dense(10, activation="tanh", name="additional_layer")
])

# Заморозка исходной модели
model.trainable = False

print("\nРасширенная модель:")
model_extended.summary()

# Компиляция расширенной модели
model_extended.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

# Дообучение только нового слоя
print("\nДообучение расширенной модели...")
history_extended = model_extended.fit(x_train, y_train_cat, 
                                     batch_size=32, 
                                     epochs=3,
                                     validation_data=(x_test, y_test_cat))

# Проверка замороженных весов
print("\nПроверка заморозки весов:")
for i, layer in enumerate(model.layers):
    print(f"Слой {i} ({layer.name}): обучается = {layer.trainable}")


Расширенная модель:


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_3 (Sequential)            │ (None, 10)                  │         101,770 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ additional_layer (Dense)             │ (None, 10)                  │             110 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 101,880 (397.97 KB)

 Trainable params: 110 (440.00 B)

 Non-trainable params: 101,770 (397.54 KB)


Дообучение расширенной модели...
Epoch 1/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 936us/step - accuracy: 0.0022 - loss: 9.2435 - val_accuracy: 0.0047 - val_loss: 11.3744
Epoch 2/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 889us/step - accuracy: 0.0019 - loss: 10.0907 - val_accuracy: 0.0051 - val_loss: 11.1182
Epoch 3/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 904us/step - accuracy: 0.0016 - loss: 11.1320 - val_accuracy: 0.0036 - val_loss: 11.0429

Проверка заморозки весов:
Слой 0 (hidden_1): обучается = False
Слой 1 (output): обучается = False


In [12]:
# Разморозка модели
model.trainable = True

# Заморозка только первого слоя
model.layers[0].trainable = False

print("\nЗаморозка только первого слоя:")
for i, layer in enumerate(model.layers):
    print(f"Слой {i} ({layer.name}): обучается = {layer.trainable}")

# Перекомпиляция и обучение
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("\nОбучение с замороженным первым слоем...")
history_partial = model.fit(x_train, y_train_cat, 
                           batch_size=32, 
                           epochs=2,
                           validation_data=(x_test, y_test_cat))


Заморозка только первого слоя:
Слой 0 (hidden_1): обучается = False
Слой 1 (output): обучается = True

Обучение с замороженным первым слоем...
Epoch 1/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 906us/step - accuracy: 0.9934 - loss: 0.0240 - val_accuracy: 0.9800 - val_loss: 0.0644
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 847us/step - accuracy: 0.9947 - loss: 0.0206 - val_accuracy: 0.9805 - val_loss: 0.0652
